In [27]:
import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm
tqdm.pandas()

#

In [28]:
MAX_DISTANCE_KM = 200 / 1000

In [29]:
# distance = geodesic(atm_coords, obj_coords).km #lat, long

In [30]:
df = pd.read_csv("concat_df.csv").drop("Unnamed: 0", axis='columns')
print(len(df))
df.head()

8461


,id,atm_group,address,address_rus,lat,long,target
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277


In [31]:
df_gps = df[['lat','long']].drop_duplicates().dropna()
len(df_gps)

6243

In [32]:
df_gps.head()

,lat,long
0,46.940995,142.738319
1,46.937353,142.753348
2,46.959413,142.741113
3,55.805827,37.515146
4,69.343541,88.211228


In [33]:
atm_n = 0
atm_loc = (df_gps.iloc[atm_n, 0], df_gps.iloc[atm_n, 1])
atm_loc

(46.940995, 142.738319)

In [34]:
df_obj = pd.read_csv("united_objs.csv") #Выгруженные объекты инфраструктуры на карте
df_obj.head()

,id,type,lon,lat,geometry,region
0,0,place::city,20.510584,54.710128,POINT (20.5105838 54.710128),kaliningrad.csv
1,1,place::town,19.894158,54.643721,POINT (19.8941584 54.6437214),kaliningrad.csv
2,2,amenity::ferry_terminal,21.130789,55.685593,POINT (21.1307891 55.6855933),kaliningrad.csv
3,3,place::town,20.475019,54.960037,POINT (20.4750188 54.9600365),kaliningrad.csv
4,4,place::suburb,20.334466,54.649849,POINT (20.3344662 54.6498486),kaliningrad.csv


In [35]:
len(df_obj)

1254879

In [36]:
#Фильтруем объекты инфраструктуры
#Калинград не смотрим, нет данных по АТМ
filter1 = df_obj[df_obj['region'] != 'kaliningrad.csv'] 
#Категорию place не смотрим, не является объектом инфраструктуры
filter2 = df_obj[~df_obj['type'].str.startswith('place::')] 


filter_by = """
shop::convenience
amenity::cafe
amenity::pharmacy
shop::supermarket
amenity::atm
amenity::bank
shop::outpost
amenity::fast_food
shop::clothes
amenity::post_office
shop::mobile_phone
shop::car_repair
shop::alcohol
shop::hairdresser
amenity::restaurant
office::company
shop::car_parts
shop::beauty
tourism::hotel
shop::hardware
shop::florist
shop::electronics
amenity::doctors
shop::furniture
amenity::dentist
shop::doityourself
shop::bakery
amenity::bar
amenity::police
shop::butcher
amenity::clinic
tourism::guest_house
amenity::library
shop::shoes
shop::cosmetics
amenity::vending_machine
shop::jewelry
amenity::post_box
shop::pet
shop::kiosk
shop::greengrocer
shop::car
amenity::community_centre
tourism::museum
shop::chemist
amenity::townhall
healthcare::laboratory
shop::optician
amenity::pub
shop::gift
shop::toys
shop::confectionery
office::lawyer
shop::books
amenity::veterinary
shop::travel_agency
shop::stationery
shop::sports
amenity::bus_station
shop::computer
shop::variety_store
office::logistics
amenity::taxi
shop::money_lender
tourism::hostel
shop::department_store
shop::seafood
shop::tobacco
amenity::parcel_locker
shop::dry_cleaning
shop::houseware
shop::copyshop
amenity::courthouse
shop::beverages
office::estate_agent
shop::outdoor
amenity::cinema
shop::baby_goods
shop::bicycle
amenity::bicycle_rental
shop::medical_supply
shop::mall
shop::fabric
shop::garden_centre
shop::tailor
tourism::motel
shop::e-cigarette
amenity::theatre
amenity::nightclub
shop::photo
amenity::marketplace
shop::farm
shop::bookmaker
amenity::arts_centre
shop::fishing
shop::erotic
office::notary
office::administrative
shop::bathroom_furnishing
shop::wine
shop::doors
office::property_management
shop::interior_decoration
shop::dairy
shop::laundry
amenity::language_school
shop::curtain
shop::electrical
office::newspaper
amenity::public_building
shop::bag
shop::kitchen
office::travel_agent
shop::pastry
shop::paint
shop::motorcycle
shop::second_hand
shop::massage
office::research
shop::tea
amenity::shower
office::it
amenity::parking_space
shop::bed
amenity::music_school
shop::craft
shop::pyrotechnics
shop::radiotechnics
amenity::lounger
office::advertising_agency
shop::coffee
shop::glaziery
shop::flooring
shop::perfumery
amenity::bureau_de_change
shop::ice_cream
amenity::studio
shop::vacant
tourism::gallery
shop::tattoo
shop::boutique
shop::sewing
shop::appliance
shop::antiques
shop::hunting
amenity::public_bath
shop::musical_instrument
shop::watches
office::educational_institution
shop::religion
office::political_party
tourism::zoo
office::architect
office::association
amenity::food_court
shop::general
amenity::boat_rental
shop::lighting
office::accountant
shop::music
office::diplomatic
shop::frame
office::government
shop::fashion
amenity::internet_cafe
office::financial
shop::tiles
shop::health_food
shop::art
""".strip("\n").split("\n")

#В переменной filter3 DataFrame c обьектами инфраструктуры
filter3 = filter2[filter2['type'].isin(filter_by)]

In [39]:
#Оставляем только координаты
filter4 = filter3[['lat', 'lon']].drop_duplicates()
filter4

,lat,lon
64,54.720887,20.499257
65,54.711600,20.507600
74,55.011921,21.243174
93,54.723725,20.490009
99,54.709813,20.555247
...,...,...
1254856,57.120704,65.501475
1254858,55.176628,61.310363
1254859,60.932509,76.623513
1254863,56.696102,60.537475


In [40]:
#

x1, y1 = 0, 0
x2, y2 = 0.01, 0


dist = geodesic((x1, y1), (x2, y2)).km

dist, dist > MAX_DISTANCE_KM

(1.1057427583286865, True)

In [52]:
%%time


def find_distance(row, atm_loc):
    if all(pd.isna(row)):
        return None
    obj_loc = (row['lat'], row['lon'])
    distance = geodesic(atm_loc, obj_loc).km
    return 0 if distance > MAX_DISTANCE_KM else 1 #distance

#Функция отбрасывает все объекты на расстоянии примерно 1км и больше
def value_checker(x, y):
    return abs(x - y) <= 0.01


def find_atm_distance(row):    
    atm_loc = (row['lat'], row['long'])
    filter_x = filter4[filter4['lat'].apply(value_checker, args=(atm_loc[0],))]
    filter_r = filter_x[filter_x['lon'].apply(value_checker, args=(atm_loc[1],))]
    distances = filter_r.reset_index().apply(find_distance, args=(atm_loc,), axis=1)
    return sum(distances)

sum_col = df_gps.progress_apply(find_atm_distance, axis=1)
sum_col

100%|█████████████████████████████████████████████████████████████████████| 6243/6243 [38:29<00:00,  2.70it/s]

CPU times: user 38min 19s, sys: 12.1 s, total: 38min 31s
Wall time: 38min 29s


0        30
1        17
2        12
3       125
4        28
       ... 
8454     17
8455     22
8456     33
8457     11
8460     70
Length: 6243, dtype: int64

In [53]:
df_gps['dist'] = sum_col
df_gps.to_csv('coords_dist_count.csv', index=False)

In [54]:
def check_if(row):
    x, y = row['lat'], row['long']
    check_against = df_gps[(df_gps['lat'] == x) & (df_gps['long'] == y)]['dist'].tolist()
    return check_against[0] if check_against else None

#x, y = df.iloc[0, 4:6]
#df.head(1) #46 lat 142 long
#df_gps[(df_gps['lat'] == x) & (df_gps['long'] == y)]['dist'].tolist()

df['dist'] = df.apply(check_if, axis=1)
df

,id,atm_group,address,address_rus,lat,long,target,dist
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500,30
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710,17
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540,12
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035,125
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277,28
...,...,...,...,...,...,...,...,...
8456,7805.0,5478.0,"D. 12, PROEZD ZAREVYI MOSKVA G","Заревый проезд, 12, Москва, Россия, 127282",55.886488,37.656829,NaN,33
8457,7395.0,5478.0,"D. 2B, UL. MIRA SOCHI G","улица Мира, 4, Адлерский район, Сочи, Краснода...",43.438310,39.912211,NaN,11
8458,3339.0,8083.0,"KARMANITSKIY, 9 Moskva","Карманицкий переулок, 9, Москва, Россия, 119002",55.748275,37.583571,NaN,124
8459,675.0,1942.0,3A SHARAPOVSKIY WAY MYTISHCHI,"Шараповская улица, 1, Мытищи, Московская облас...",55.910992,37.748466,NaN,5


In [49]:
check_nan = df[df['dist'].isna()]
check_nan['lat'].drop_duplicates().tolist(), check_nan['long'].drop_duplicates().tolist()

([], [])

In [56]:
# df[~df['lat'].isna()]
df #.dropna()

,id,atm_group,address,address_rus,lat,long,target,dist
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500,30
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710,17
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540,12
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035,125
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277,28
...,...,...,...,...,...,...,...,...
8456,7805.0,5478.0,"D. 12, PROEZD ZAREVYI MOSKVA G","Заревый проезд, 12, Москва, Россия, 127282",55.886488,37.656829,NaN,33
8457,7395.0,5478.0,"D. 2B, UL. MIRA SOCHI G","улица Мира, 4, Адлерский район, Сочи, Краснода...",43.438310,39.912211,NaN,11
8458,3339.0,8083.0,"KARMANITSKIY, 9 Moskva","Карманицкий переулок, 9, Москва, Россия, 119002",55.748275,37.583571,NaN,124
8459,675.0,1942.0,3A SHARAPOVSKIY WAY MYTISHCHI,"Шараповская улица, 1, Мытищи, Московская облас...",55.910992,37.748466,NaN,5


In [57]:
df.to_csv('df_dist_count.csv', index=False)